In [1]:
#######################################################
##### STRIP TWEET #####################################
#######################################################

#import pandas as pd
#df = pd.read_csv("train2017.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
#corpus = df[3].tolist()

import nltk
import string
import re

f = open("train2017.tsv")
line = f.readline()

tweets = []
translate_table = dict((ord(char), None) for char in string.punctuation)   
cnt = 1
while line:
    pretext = ' '.join(line.split()[3:])
    pretext = ' '.join(word for word in pretext.split(' ') if not word.startswith('@')) #remove @user
    pretext = re.sub(r"http\S+", "", pretext) #remove link
    pretext = pretext.translate(translate_table) #remove symbols 
    #remove numbers??
    
    tweet = pretext.lower()
    tweets.append(tweet)
    line = f.readline()
    cnt += 1

#print(tweets)

In [2]:
#######################################################
##### TOKENIZATION ####################################
#######################################################

from nltk import word_tokenize

tokens = []
for tweet in tweets:
    token = []
    token = word_tokenize(tweet)
    tokens.append(token)

#print(tokens)

In [3]:
from nltk import pos_tag

from nltk.corpus import wordnet
from nltk.stem import  WordNetLemmatizer
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

tweets = []
for token_list in tokens:
    lemmatized = []
    for word in token_list:
        lemmatized.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    tweets.append(lemmatized)

#print(tweets)

In [4]:
corpus = []
for tweet in tweets:
    corpus.append(" ".join(str(word) for word in tweet))

#print(corpus)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.1)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray()[0])

['and', 'at', 'be', 'for', 'go', 'have', 'in', 'it', 'may', 'my', 'of', 'on', 'the', 'to', 'tomorrow', 'with', 'you']
[0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0]
